In [30]:
from rfpipe import candidates

import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u

from astropy import coordinates as coord
import logging
logger = logging.getLogger()
logger = logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(threadName)s - %(levelname)s -'
                                                        ' %(message)s')

In [24]:
def query_psrcat(cd, radius, psrcat_loc):
    """ Takes canddata of the candidate, radius in arcminnutes (in astropy.coordinates)  and 
    location of psrcat (csv) as input and returns a dataframe of the pulsars found within
    the specified radius of the candidate. 
    """

    l1 = cd.l1
    m1 = cd.m1
    pt_ra, pt_dec = cd.state.metadata.radec
    src_ra_deg, src_dec_deg = candidates.source_location(pt_ra, pt_dec, l1, m1)
    center = SkyCoord(src_ra_deg + ' ' + src_dec_deg, unit=(u.hourangle, u.deg), frame='icrs')
    
    radius = coord.Angle(radius)    
    r_deg = radius.to('deg').value

    ra = center.ra.deg
    dec = center.dec.deg
    
    ra_min = ra - r_deg
    ra_max = ra + r_deg
    dec_min = dec - r_deg
    dec_max = dec + r_deg
    
    psrcat_df = pd.read_csv(psrcat_loc, skiprows=2, usecols = [1, 3, 6, 9, 10],  \
                        names = ['Name', 'DM', 'P0', 'RAJD', 'DECJD'])
    
    psrcat_region_mask = (psrcat_df['RAJD'] > ra_min) & (psrcat_df['RAJD'] < ra_max) & \
                         (psrcat_df['DECJD'] > dec_min) & (psrcat_df['DECJD'] < dec_max)
    
    rt_psrcat = psrcat_df[psrcat_region_mask]
    logging.info(f'Found {len(rt_psrcat)} pulsars in PSRCAT within {radius} of (ra = {ra:.4f}deg, dec = {dec:.4f}deg')
    return rt_psrcat

In [31]:
def query_frbcat(cd, radius, frbcat_loc):
    """ Takes canddata of the candidate, radius in arcminnutes (in astropy.coordinates)  and 
    location of frbcat (csv) as input and returns a dataframe of the frbs found within
    the specified radius of the candidate. 
    """

    l1 = cd.l1
    m1 = cd.m1
    st = cd.state
    pt_ra, pt_dec = st.metadata.radec
    src_ra_deg, src_dec_deg = candidates.source_location(pt_ra, pt_dec, l1, m1)
    center = SkyCoord(src_ra_deg + ' ' + src_dec_deg, unit=(u.hourangle, u.deg), frame='icrs')
    
    radius = coord.Angle(radius)    
    r_deg = radius.to('deg').value

    ra = center.ra.deg
    dec = center.dec.deg
    
    ra_min = ra - r_deg
    ra_max = ra + r_deg
    dec_min = dec - r_deg
    dec_max = dec + r_deg
    
    frbcat_df = pd.read_csv(frbcat_loc)    
    ra_deg = []
    dec_deg = []
    for idx, row in frbcat_df.iterrows():
        center = SkyCoord(l=row['rop_gl']*u.deg, b=row['rop_gb']*u.deg, frame='galactic')
        ra_deg.append(center.icrs.ra.deg)
        dec_deg.append(center.icrs.dec.deg)
        
    frbcat_df['RAJD'] = ra_deg
    frbcat_df['DECJD'] = dec_deg    
    
    frbcat_region_mask = (frbcat_df['RAJD'] > ra_min) & (frbcat_df['RAJD'] < ra_max) & \
                         (frbcat_df['DECJD'] > dec_min) & (frbcat_df['DECJD'] < dec_max)
    rt_frbcat = frbcat_df[frbcat_region_mask]
    
    logging.info(f'Found {len(rt_frbcat)} pulsars in FRBCAT within {radius} of (ra = {ra:.4f}deg, dec = {dec:.4f}deg')
    return rt_frbcat

In [32]:
file = '../cands_16A-459_TEST_1hr_000.57633.66130137732.scan7.cut.7.1.pkl'
cds = list(candidates.iter_cands(file))

In [33]:
query_psrcat(cds[0], 500*u.arcminute, '../psrcat_output.csv')

2019-10-06 20:14:55,789 - root - INFO - Found 7 pulsars in PSRCAT within 500.0 arcmin of (ra = 83.3210deg, dec = 32.9303deg


,Name,DM,P0,RAJD,DECJD
171,J0509+3801,69.0794,0.07654134872199,77.38245,38.02169
197,J0538+2817,39.570,0.14315825891177,84.60440,28.28588
198,J0540+3207,61.97,0.5242708632083,85.15465,32.12703
206,J0546+2441,73.81,2.84385038524,86.61983,24.68917
208,J0554+3107,*,0.464960965736,88.52088,31.12806
209,J0555+3948,37,1.1469058,88.75000,39.80000
215,J0605+3757,20.9462,0.0027279541573939,91.27310,37.95999


In [34]:
query_frbcat(cds[0], 500*u.arcminute, '../frbcat.csv')

2019-10-06 20:14:58,185 - root - INFO - Found 1 pulsars in FRBCAT within 500.0 arcmin of (ra = 83.3210deg, dec = 32.9303deg


,frb_name,utc,telescope,rop_raj,rop_decj,rop_gl,rop_gb,rmp_dm,rmp_width,rmp_snr,RAJD,DECJD
55,FRB121102,2012/11/02 06:35:53.244,arecibo,05:32:09,33:05:13,174.95,-0.225138,557&plusmn2,3.0,14.0,83.037331,33.087154
